In [1]:
import os, sys
import glob

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

import datetime

In [3]:
# preprocess weather 정보
# df_weather = pd.DataFrame()
# for fname in os.listdir("./weather_hour/"):
#     df_tmp = pd.read_csv(f"./weather_hour/{fname}", encoding="cp949")
#     if df_weather.empty:
#         df_weather = df_tmp.copy()
#     else:
#         df_weather = df_weather.append(df_tmp)
# df_weather = df_weather.reset_index(drop=True)
# df_weather.loc[:, '일시'] = df_weather.loc[:, '일시'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M"))
# df_weather.to_csv("./database/weather_all.csv")

# fillna
- 날씨마루에서 제공하는 결측치 보간 방법 사용
- https://bd.kma.go.kr/kma2020/dta/edu/KBP57200_Python.do

In [17]:
df_weather = pd.read_csv("./database/weather_all.csv", index_col=[0])

In [20]:
cond = ~(df_weather.isna().sum(axis=1) == 0)
df_weather.loc[cond]

,지점,지점명,일시,기온(°C),강수량(mm),풍속(m/s),습도(%),현지기압(hPa),일조(hr),적설(cm),전운량(10분위)
0,108,서울,2015-01-01 00:00:00,NaN,NaN,6.1,NaN,NaN,NaN,NaN,NaN
1,108,서울,2015-01-01 01:00:00,-7.4,NaN,4.7,33.0,1011.8,NaN,NaN,NaN
2,108,서울,2015-01-01 02:00:00,-8.0,NaN,4.5,35.0,1011.7,NaN,NaN,NaN
3,108,서울,2015-01-01 03:00:00,-8.4,NaN,3.8,37.0,1012.1,NaN,NaN,0.0
4,108,서울,2015-01-01 04:00:00,-8.8,NaN,4.9,35.0,1012.3,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...
43819,108,서울,2019-12-31 19:00:00,-6.9,NaN,1.7,38.0,1024.1,NaN,NaN,0.0
43820,108,서울,2019-12-31 20:00:00,-7.0,NaN,2.9,41.0,1024.1,NaN,NaN,0.0
43821,108,서울,2019-12-31 21:00:00,-7.4,NaN,0.8,43.0,1023.9,NaN,NaN,0.0
43822,108,서울,2019-12-31 22:00:00,-8.2,NaN,0.0,42.0,1023.3,NaN,NaN,0.0


In [21]:
# '강수량(mm)', '적설(cm)' 는 0으로 치환
df_weather['강수량(mm)'] = df_weather['강수량(mm)'].fillna(value = 0)
df_weather['적설(cm)'] = df_weather['강수량(mm)'].fillna(value = 0)

# '현지기압(hPa)', 는 ffill, bfill
df_weather['현지기압(hPa)'] = df_weather['현지기압(hPa)'].fillna(method='ffill')
df_weather['현지기압(hPa)'] = df_weather['현지기압(hPa)'].fillna(method='bfill')

In [22]:
for col in ['기온(°C)', '풍속(m/s)', '습도(%)', '일조(hr)', '전운량(10분위)']:
    df_weather[col] = df_weather[col].fillna(method='ffill')
    df_weather[col] = df_weather[col].fillna(method='bfill')

In [23]:
df_weather.loc[cond]

,지점,지점명,일시,기온(°C),강수량(mm),풍속(m/s),습도(%),현지기압(hPa),일조(hr),적설(cm),전운량(10분위)
0,108,서울,2015-01-01 00:00:00,-7.4,0.0,6.1,33.0,1011.8,0.0,0.0,0.0
1,108,서울,2015-01-01 01:00:00,-7.4,0.0,4.7,33.0,1011.8,0.0,0.0,0.0
2,108,서울,2015-01-01 02:00:00,-8.0,0.0,4.5,35.0,1011.7,0.0,0.0,0.0
3,108,서울,2015-01-01 03:00:00,-8.4,0.0,3.8,37.0,1012.1,0.0,0.0,0.0
4,108,서울,2015-01-01 04:00:00,-8.8,0.0,4.9,35.0,1012.3,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
43819,108,서울,2019-12-31 19:00:00,-6.9,0.0,1.7,38.0,1024.1,0.1,0.0,0.0
43820,108,서울,2019-12-31 20:00:00,-7.0,0.0,2.9,41.0,1024.1,0.1,0.0,0.0
43821,108,서울,2019-12-31 21:00:00,-7.4,0.0,0.8,43.0,1023.9,0.1,0.0,0.0
43822,108,서울,2019-12-31 22:00:00,-8.2,0.0,0.0,42.0,1023.3,0.1,0.0,0.0


In [24]:
df_weather.to_csv("./final_db/df_weather.csv")

전체 db랑 합쳐주자

In [117]:
import datetime

In [139]:
df_data = pd.read_csv("./final_db/data.csv", index_col=[0])

/Users/bowoojang/Desktop/idk/venv/univ_env/lib/python3.8/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [140]:
for weather_col in list(df_weather.columns[3:]):
    df_data[weather_col] = np.nan

In [143]:
drop_24_idx = df_data.loc[df_data['time']==24].index

In [145]:
df_data = df_data.drop(index=drop_24_idx)

In [146]:
for date in df_data['date'].unique():
    for time in df_data['time'].unique():
        time = int(time)
        cond = (df_data['date']==date) & (df_data['time']== time)
        
        when = datetime.datetime.strptime(f"{date} {time}:00:00", "%Y-%m-%d %H:%M:%S")
        
        df_tmp = df_weather.loc[df_weather['일시']==str(when), '기온(°C)':].reset_index(drop=True)
        df_data.loc[cond, '기온(°C)':] = df_tmp

In [149]:
df_data_weather = df_data.copy()

In [151]:
df_data_weather.fillna(method='ffill')

,holiday,date,time,line,order,from_stt,to_stt,in,out,crowd,기온(°C),강수량(mm),풍속(m/s),습도(%),현지기압(hPa),일조(hr),적설(cm),전운량(10분위)
0,2,2015-01-01,5,1호선,상선,서울역,시청,441.0,392.0,4.0,-9.1,0.0,5.7,35.0,1011.8,0.0,0.0,0.0
1,2,2015-01-01,5,1호선,상선,시청,종각,101.0,57.0,5.0,-9.1,0.0,5.7,35.0,1011.8,0.0,0.0,0.0
2,2,2015-01-01,5,1호선,상선,종각,종로3가,898.0,60.0,6.0,-9.1,0.0,5.7,35.0,1011.8,0.0,0.0,0.0
3,2,2015-01-01,5,1호선,상선,종로3가,종로5가,371.0,27.0,6.0,-9.1,0.0,5.7,35.0,1011.8,0.0,0.0,0.0
4,2,2015-01-01,5,1호선,상선,종로5가,동대문,57.0,22.0,6.0,-9.1,0.0,5.7,35.0,1011.8,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1605885,0,2015-12-31,23,4호선,하선,쌍문,수유,341.0,1423.0,13.0,-9.1,0.0,5.7,35.0,1011.8,0.0,0.0,0.0
1605886,0,2015-12-31,23,4호선,하선,창동,쌍문,499.0,1469.0,13.0,-9.1,0.0,5.7,35.0,1011.8,0.0,0.0,0.0
1605887,0,2015-12-31,23,4호선,하선,노원,창동,747.0,1463.0,11.0,-9.1,0.0,5.7,35.0,1011.8,0.0,0.0,0.0
1605888,0,2015-12-31,23,4호선,하선,상계,노원,237.0,984.0,3.0,-9.1,0.0,5.7,35.0,1011.8,0.0,0.0,0.0


In [152]:
df_data_weather.to_csv("./final_db/data_w_weather.csv")

In [6]:
# utils: 앞뒤값 살펴보기
na_col = '기온(°C)'
# ['지점', '지점명', '일시', '기온(°C)', '강수량(mm)', '풍속(m/s)', 
# '습도(%)', '현지기압(hPa)', '일조(hr)', '적설(cm)', '전운량(10분위)']

na_idx = df_weather.loc[df_weather[na_col].isna()].index
fna_idx = []

for idx in na_idx:
    fna_idx += [ i for i in range(idx-3,idx+3)]

fna_idx = sorted(list(set(fna_idx)))
fna_idx = [x for x in fna_idx if x>=0 and x<= df_weather.index.max()]
df_weather.iloc[fna_idx, :]

,지점,지점명,일시,기온(°C),강수량(mm),풍속(m/s),습도(%),현지기압(hPa),일조(hr),적설(cm),전운량(10분위)
0,108,서울,2015-01-01 00:00:00,NaN,0.0,6.1,NaN,1011.8,NaN,0.0,NaN
1,108,서울,2015-01-01 01:00:00,-7.4,0.0,4.7,33.0,1011.8,NaN,0.0,NaN
2,108,서울,2015-01-01 02:00:00,-8.0,0.0,4.5,35.0,1011.7,NaN,0.0,NaN
15206,108,서울,2017-09-26 14:00:00,30.7,0.0,1.8,37.0,1001.1,1.0,0.0,5.0
15207,108,서울,2017-09-26 15:00:00,31.1,0.0,1.5,39.0,1000.3,1.0,0.0,6.0
15208,108,서울,2017-09-26 16:00:00,30.4,0.0,1.2,39.0,1000.0,0.7,0.0,8.0
15209,108,서울,2017-09-26 17:00:00,NaN,0.0,NaN,NaN,1000.0,NaN,0.0,10.0
15210,108,서울,2017-09-26 18:00:00,27.9,0.0,NaN,45.0,1000.0,0.0,0.0,10.0
15211,108,서울,2017-09-26 19:00:00,26.4,0.0,NaN,60.0,1000.4,0.0,0.0,NaN
15588,108,서울,2017-10-12 12:00:00,13.1,0.0,1.7,65.0,1010.4,0.0,0.0,10.0


In [ ]:
# 기상청에서 이렇게 하더만 안되겠구만... 그냥 interpolate 하자.. 
# import sklearn.neighbors._base
# sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base

# from missingpy import KNNImputer


# print(df_weather[na_col].isnull().sum())
# temp_idx = df_weather.loc[df_weather[na_col].isnull(), :].index
# df_weather.loc[df_weather[na_col].isnull(), :]


# # '기온(°C)', '풍속(m/s)', '습도(%)' 
# # KnnImputation을 활용하여 결측치 처리
# imputer = KNNImputer(n_neighbors = 20)
# df_weather[na_col] = pd.DataFrame(imputer.fit_transform(df_weather[na_col].to_numpy().reshape(-1, 1)))
# df_weather[na_col].isnull().sum()
# df_weather.iloc[temp_idx, :]
